In [29]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [30]:
# Define constants
input_shape = (224, 224, 3)
num_classes = 5
batch_size = 32
epochs = 10

# Data loading and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.10  # Splitting the data into training and validation sets
)

train_generator = train_datagen.flow_from_directory(
    'D:\\Btech_AI\\Projects\\Gesture-recognition\\Dataset\\train',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)


validation_generator = train_datagen.flow_from_directory(
    'D:\\Btech_AI\\Projects\\Gesture-recognition\\Dataset\\train',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 2140 images belonging to 5 classes.
Found 235 images belonging to 5 classes.


### Without K-fold cross Validation

In [31]:
# Model building
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:
# Training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)
# Evaluate the model on the entire training set
train_loss, train_accuracy = model.evaluate(train_generator, verbose=1)

# Evaluate the model on the entire validation set
val_loss, val_accuracy = model.evaluate(validation_generator, verbose=1)

# Print the results
print(f"Training Loss: {train_loss}")
print(f"Training Accuracy: {train_accuracy}")
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
66/66 [==============================] - 30s 397ms/step - loss: 0.3398 - accuracy: 0.9051 - val_loss: 0.0481 - val_accuracy: 0.9911
Epoch 2/10
66/66 [==============================] - 26s 388ms/step - loss: 0.0387 - accuracy: 0.9862 - val_loss: 0.0643 - val_accuracy: 0.9777
Epoch 3/10
66/66 [==============================] - 25s 382ms/step - loss: 0.0358 - accuracy: 0.9862 - val_loss: 0.0392 - val_accuracy: 0.9911
Epoch 4/10
66/66 [==============================] - 26s 387ms/step - loss: 0.0369 - accuracy: 0.9867 - val_loss: 0.1402 - val_accuracy: 0.9598
Epoch 5/10
66/66 [==============================] - 26s 390ms/step - loss: 0.0325 - accuracy: 0.9886 - val_loss: 0.1009 - val_accuracy: 0.9598
Epoch 6/10
66/66 [==============================] - 25s 383ms/step - loss: 0.0328 - accuracy: 0.9896 - val_loss: 0.0237 - val_accuracy: 0.9866
Epoch 7/10
66/66 [==============================] - 26s 387ms/step - loss: 0.0117 - accuracy: 0.9957 - val_loss: 0.0433 - val_accuracy: 0.9821

In [33]:
# Save the trained model
model.save('gesture_recognition_model.h5')

In [34]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'D:\\Btech_AI\\Projects\\Gesture-recognition\\Dataset\\test',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Set shuffle to False for the testing set
)

Found 265 images belonging to 5 classes.
